In [1]:
%pip install -q sec-api

Note: you may need to restart the kernel to use updated packages.


In [1]:
API_KEY = 'YOUR_API_KEY'

In [2]:
from sec_api import QueryApi

queryApi = QueryApi(api_key=API_KEY)

In [3]:
import pandas as pd
import numpy as np

In [8]:
df = pd.read_excel('/Users/abhishekthakur/Desktop/Katbotz/top100_MCap_nasdaq_screener.xlsx')
print(df.head(10))

  Symbol                                         Name Last Sale Net Change  \
0   ABBV                     AbbVie Inc. Common Stock     162.4      -1.38   
1    ABT             Abbott Laboratories Common Stock    113.92       0.42   
2   ACHC  Acadia Healthcare Company Inc. Common Stock     85.86       1.17   
3    ALC                   Alcon Inc. Ordinary Shares     80.05       0.36   
4   ALGN           Align Technology Inc. Common Stock    271.64      -4.72   
5   ALNY    Alnylam Pharmaceuticals Inc. Common Stock    193.39      -1.69   
6   AMGN                      Amgen Inc. Common Stock    306.51       3.41   
7   APLS    Apellis Pharmaceuticals Inc. Common Stock     70.23       4.14   
8   ARGX         argenx SE American Depositary Shares    373.67       -7.6   
9    AZN   AstraZeneca PLC American Depositary Shares     69.42       0.24   

  % Change      Market Cap         Country IPO Year     Volume       Sector  \
0 -0.00843  286723277170.0   United States   2012.0  4970934.0

In [17]:


# Assuming 'df' is your DataFrame and 'column name' is the name of the column
unique_values = df['Symbol'].unique()

# Convert the unique values to a comma-separated string
unique_values_string = ', '.join(map(str, unique_values))

# Create the ticker_query string
ticker_query = f'ticker:({unique_values_string})'

# Display the resulting query
print(ticker_query)


ticker:(ABBV, ABT, ACHC, ALC, ALGN, ALNY, AMGN, APLS, ARGX, AZN, BAX, BDX, BGNE, BIIB, BKDT, BMRN, BMY, BNTX, BSX, BTI, CAH, CERE, CHE, CI, CNC, COO, COR, CRL, CTLT, CYTK, DGX, DVA, DXCM, ELV, EW, EXAS, FMS, GILD, GMED, GSK, HCA, HOLX, HSIC, HUM, ICLR, IDXX, ILMN, IMGN, INCY, IQV, ISRG, JNJ, KRTX, LEGN, LH, LLY, MCK, MDT, MEDP, MMM, MO, MOH, MRK, MRNA, NBIX, NVO, NVS, PEN, PFE, PHG, PM, PODD, QGEN, RDY, REGN, RGEN, RMD, ROIV, RPRX, SNN, SNY, SRPT, STE, SWAV, SYK, Symbol, TAK, TECH, TEVA, TFX, THC, UHS, UNH, UTHR, VRTX, VTRS, WST, XRAY, ZBH, ZTS)


In [18]:
def get_filings(query):
  from_param = 0
  size_param = 200
  all_filings = []

  while True:
    query['from'] = from_param
    query['size'] = size_param
    
    response = queryApi.get_filings(query)
    filings = response['filings']

    if len(filings) == 0:
      break

    all_filings.extend(filings)

    from_param += size_param

  return pd.json_normalize(all_filings)


form_type_query  = 'formType:("10-K", "10-Q") AND NOT formType:("10-K/A", "10-Q/A", NT)'
ticker_query     = ticker_query
date_range_query = 'filedAt:[2023-01-01 TO 2024-01-12]'

lucene_query = f"{form_type_query} AND {ticker_query} AND {date_range_query}"

search_query = {
  "query": { "query_string": { 
      "query": lucene_query,
  } },
  "from": "0",
  "size": "200",
  "sort": [{ "filedAt": { "order": "desc" } }]
}

filings = get_filings(search_query)

In [22]:
filings.tail(10)

,id,accessionNo,cik,ticker,companyName,companyNameLong,formType,description,filedAt,linkToTxt,linkToHtml,linkToXbrl,linkToFilingDetails,entities,documentFormatFiles,dataFiles,seriesAndClassesContractsInformation,periodOfReport
314,7c74d7aee6dc6d6701c523ed0a884266,0001558370-23-000979,842023,TECH,BIO-TECHNE Corp,BIO-TECHNE Corp (Filer),10-Q,Form 10-Q - Quarterly report [Sections 13 or 1...,2023-02-07T16:30:38-05:00,https://www.sec.gov/Archives/edgar/data/842023...,https://www.sec.gov/Archives/edgar/data/842023...,,https://www.sec.gov/Archives/edgar/data/842023...,"[{'companyName': 'BIO-TECHNE Corp (Filer)', 'c...","[{'sequence': '1', 'description': '10-Q', 'doc...","[{'sequence': '6', 'description': 'EX-101.SCH'...",[],2022-12-31
315,8cc2ffb5994974f8aabffe4a3a3a723c,0000879169-23-000008,879169,INCY,INCYTE CORP,INCYTE CORP (Filer),10-K,Form 10-K - Annual report [Section 13 and 15(d...,2023-02-07T16:02:05-05:00,https://www.sec.gov/Archives/edgar/data/879169...,https://www.sec.gov/Archives/edgar/data/879169...,,https://www.sec.gov/Archives/edgar/data/879169...,"[{'companyName': 'INCYTE CORP (Filer)', 'cik':...","[{'sequence': '1', 'description': '10-K', 'doc...","[{'sequence': '8', 'description': 'XBRL TAXONO...",[],2022-12-31
316,4c02b50d0701e9e8f192eef2ea7e7ee7,0001596783-23-000017,1596783,CTLT,"Catalent, Inc.","Catalent, Inc. (Filer)",10-Q,Form 10-Q - Quarterly report [Sections 13 or 1...,2023-02-07T09:26:06-05:00,https://www.sec.gov/Archives/edgar/data/159678...,https://www.sec.gov/Archives/edgar/data/159678...,,https://www.sec.gov/Archives/edgar/data/159678...,"[{'companyName': 'Catalent, Inc. (Filer)', 'ci...","[{'sequence': '1', 'description': '10-Q', 'doc...","[{'sequence': '6', 'description': 'XBRL TAXONO...",[],2022-12-31
317,870d9bfc715bf36167c6359d90985926,0001804220-23-000008,872589,REGN,"REGENERON PHARMACEUTICALS, INC.","REGENERON PHARMACEUTICALS, INC. (Filer)",10-K,Form 10-K - Annual report [Section 13 and 15(d...,2023-02-06T09:23:24-05:00,https://www.sec.gov/Archives/edgar/data/872589...,https://www.sec.gov/Archives/edgar/data/872589...,,https://www.sec.gov/Archives/edgar/data/872589...,"[{'companyName': 'REGENERON PHARMACEUTICALS, I...","[{'sequence': '1', 'description': '10-K', 'doc...","[{'sequence': '7', 'description': 'XBRL TAXONO...",[],2022-12-31
318,7977f8b6916e874a2a36791f4e01fbbd,0000010795-23-000030,10795,BDX,BECTON DICKINSON & CO,BECTON DICKINSON & CO (Filer),10-Q,Form 10-Q - Quarterly report [Sections 13 or 1...,2023-02-02T17:03:33-05:00,https://www.sec.gov/Archives/edgar/data/10795/...,https://www.sec.gov/Archives/edgar/data/10795/...,,https://www.sec.gov/Archives/edgar/data/10795/...,[{'companyName': 'BECTON DICKINSON & CO (Filer...,"[{'sequence': '1', 'description': '10-Q', 'doc...","[{'sequence': '5', 'description': 'XBRL TAXONO...",[],2022-12-31
319,a1a8e68489575e843076868d497de3d6,0000721371-23-000007,721371,CAH,CARDINAL HEALTH INC,CARDINAL HEALTH INC (Filer),10-Q,Form 10-Q - Quarterly report [Sections 13 or 1...,2023-02-02T16:19:17-05:00,https://www.sec.gov/Archives/edgar/data/721371...,https://www.sec.gov/Archives/edgar/data/721371...,,https://www.sec.gov/Archives/edgar/data/721371...,[{'companyName': 'CARDINAL HEALTH INC (Filer)'...,"[{'sequence': '1', 'description': '10-Q', 'doc...","[{'sequence': '8', 'description': 'XBRL TAXONO...",[],2022-12-31
320,fc77e1977293df4c6efe721e8018ff57,0000927653-23-000017,927653,MCK,MCKESSON CORP,MCKESSON CORP (Filer),10-Q,Form 10-Q - Quarterly report [Sections 13 or 1...,2023-02-01T17:36:25-05:00,https://www.sec.gov/Archives/edgar/data/927653...,https://www.sec.gov/Archives/edgar/data/927653...,,https://www.sec.gov/Archives/edgar/data/927653...,"[{'companyName': 'MCKESSON CORP (Filer)', 'cik...","[{'sequence': '1', 'description': '10-Q', 'doc...","[{'sequence': '5', 'description': 'XBRL TAXONO...",[],2022-12-31
321,7f06f52b7a2b430eddc917ec005b55ee,0000859737-23-000004,859737,HOLX,HOLOGIC INC,HOLOGIC INC (Filer),10-Q,Form 10-Q - Quarterly report [Sections 13 o

In [23]:
urls = filings[['ticker', 
                'formType', 
                'periodOfReport',
                'filedAt', 
                'linkToFilingDetails']].rename(columns={'linkToFilingDetails': 'filingUrl'})

urls['financialReportsUrl'] = urls['filingUrl'].apply(lambda url: '/'.join(url.split('/')[:-1]) + '/Financial_Report.xlsx')

urls.head(10)

,ticker,formType,periodOfReport,filedAt,filingUrl,financialReportsUrl
0,CTLT,10-Q,2023-09-30,2023-12-08T17:06:39-05:00,https://www.sec.gov/Archives/edgar/data/159678...,https://www.sec.gov/Archives/edgar/data/159678...
1,CTLT,10-K,2023-06-30,2023-12-08T17:05:20-05:00,https://www.sec.gov/Archives/edgar/data/159678...,https://www.sec.gov/Archives/edgar/data/159678...
2,COO,10-K,2023-10-31,2023-12-08T16:15:18-05:00,https://www.sec.gov/Archives/edgar/data/711404...,https://www.sec.gov/Archives/edgar/data/711404...
3,MDT,10-Q,2023-10-27,2023-11-30T16:16:21-05:00,https://www.sec.gov/Archives/edgar/data/161310...,https://www.sec.gov/Archives/edgar/data/161310...
4,HSIC,10-Q,2023-09-30,2023-11-28T16:01:50-05:00,https://www.sec.gov/Archives/edgar/data/100022...,https://www.sec.gov/Archives/edgar/data/100022...
5,BDX,10-K,2023-09-30,2023-11-21T17:08:11-05:00,https://www.sec.gov/Archives/edgar/data/10795/...,https://www.sec.gov/Archives/edgar/data/10795/...
6,COR,10-K,2023-09-30,2023-11-21T16:21:51-05:00,https://www.sec.gov/Archives/edgar/data/114085...,https://www.sec.gov/Archives/edgar/data/114085...
7,HOLX,10-K,2023-09-30,2023-11-21T08:17:06-05:00,https://www.sec.gov/Archives/edgar/data/859737...,https://www.sec.gov/Archives/edgar/data/859737...
8,ROIV,10-Q,2023-09-30,2023-11-13T07:16:07-05:00,https://www.sec.gov/Archives/edgar/data/163508...,https://www.sec.gov/Archives/edgar/data/163508...
9,ILMN,10-Q,2023-10-01,2023-11-09T18:30:59-05:00,https://www.sec.gov/Archives/edgar/data/111080...,https://www.sec.gov/Archives/edgar/data/111080...


In [24]:
import os, requests

folder_path = '/Users/abhishekthakur/Desktop/Katbotz/Top_100_Healthcare/'

if not os.path.exists(folder_path):
  os.makedirs(folder_path)


def download_report(filing):
  reports_path = filing['financialReportsUrl'].replace('https://www.sec.gov/Archives/edgar/data/', '')
  base_url = 'https://archive.sec-api.io/' + reports_path
  render_api_url = base_url + '?token=' + API_KEY

  response = requests.get(render_api_url)

  file_name = f"{filing['ticker']}-{filing['periodOfReport']}-{filing['formType']}.xlsx"
  file_path = f"{folder_path}/{file_name}"

  output = open(file_path, 'wb')
  output.write(response.content)
  output.close()

In [25]:
%pip install -q pandarallel       
from pandarallel import pandarallel

number_of_workers = 10
pandarallel.initialize(progress_bar=True, nb_workers=number_of_workers, verbose=0) 

Note: you may need to restart the kernel to use updated packages.


In [26]:
urls.parallel_apply(download_report, axis=1)
print(f"✅ Downloaded {len(urls)} reports")

✅ Downloaded 324 reports
